In [469]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import torch

d:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [470]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
        array.append(random.randint(0, 1))
    return array
 
# 生成长度为10的随机数组
random_array = generate_random_array(10)
print(random_array)

[1, 0, 0, 0, 1, 0, 0, 1, 0, 1]


### 读取数据

In [471]:
X_raw=np.load('data/all.npy')

In [472]:
X_raw.shape

(8784, 18, 103)

### X train

In [473]:
j=14
train_data=X_raw[:,j,:]

In [474]:
columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(train_data, missing_columns, axis=1)
result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [475]:
missing_columns

array([  4,  15,  28,  34,  45,  47,  55,  60,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  96,  98,  99, 100, 102],
      dtype=int64)

In [476]:
# result=train_data
result.shape

(8784, 77)

In [97]:
# mean = result.mean(axis=0)
# std = result.std(axis=0)
# result = (result - mean) / (std + 1e-6)

In [477]:
# train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70]
train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
test_list=[52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]
# test_list=[71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98]

In [478]:
train_data=result[:,train_list]
test_data=result[:,test_list]

In [479]:
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

In [ ]:
# train_data = np.transpose(train_data, (0, 2, 1))
# test_data = np.transpose(test_data, (0, 2, 1))
# train_data = np.reshape(train_data, (8784, 63,1))
# test_data = np.reshape(test_data, (8784, 40,1))

In [480]:
train_data.shape

(8784, 52, 1)

In [481]:
test_data.shape

(8784, 25, 1)

### X fusion

In [294]:
# result = np.delete(train_data, missing_columns, axis=1)
# X_raw.shape

(8784, 18, 103)

In [482]:
# j=5
# train_data=X_raw[:,j,:]
train_fusion_data=np.delete(X_raw, missing_columns, axis=2)
# train_fusion_data[train_fusion_data == 0] = 0.00001
# train_fusion_data=np.nan_to_num(train_fusion_data, nan=0)
# train_fusion_data=X_raw
# train_data = np.nan_to_num(train_data, nan=0.0)

In [ ]:
# train_fusion_data[train_fusion_data == 0] = 0.00001

In [483]:
train_fusion_data.shape

(8784, 18, 77)

In [484]:
columns_to_remove = [11, 15, 16 ,17]
train_fusion_data = np.delete(train_fusion_data, columns_to_remove, axis=1)
train_fusion_data.shape

(8784, 14, 77)

In [64]:
# for i in range(5,train_fusion_data.shape[1]):
#     nan_count = np.sum(np.isnan(train_fusion_data[:,i,:]))
#     print(f"Number of zeros in the 6th feature: {nan_count}")

Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 17568
Number of zeros in the 6th feature: 254736
Number of zeros in the 6th feature: 254736
Number of zeros in the 6th feature: 509472
Number of zeros in the 6th feature: 26352
Number of zeros in the 6th feature: 17568
Number of zeros in the 6th feature: 755424


### nor

In [485]:
for feature_index in range(5, train_fusion_data.shape[1]):
    feature_column = train_fusion_data[:, feature_index, :]
    min=np.nanmin(feature_column)
    max=np.nanmax(feature_column)
    norm_data=(feature_column - min) / (max - min)
    train_fusion_data[:, feature_index, :]=norm_data
    

In [486]:
train_fusion_data[train_fusion_data == 0] = 0.00001
train_fusion_data=np.nan_to_num(train_fusion_data, nan=0)

In [390]:
# for i in range(5,train_fusion_data.shape[1]):
#     nan_count = np.sum(np.isnan(train_fusion_data[:,i,:]))
#     print(f"Number of zeros in the 6th feature: {nan_count}")

Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0
Number of zeros in the 6th feature: 0


In [36]:

# # train_fusion_data[:,5,:]
# nan_count = np.sum(np.isnan(norm_data))
# print(f"Number of zeros in the 6th feature: {nan_count}")

Number of zeros in the 6th feature: 254736


In [53]:
# columns_with_missing_data = np.any(np.isnan(train_data), axis=0)
# missing_columns = np.where(columns_with_missing_data)[0]
# result = np.delete(train_data, missing_columns, axis=2)
# # result = (result - np.min(result)) / (np.max(result) - np.min(result))

In [54]:
# unique_arr = np.unique(missing_columns)
# unique_arr

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17], dtype=int64)

In [487]:
result=train_fusion_data
result

array([[[2.01200000e+03, 2.01200000e+03, 2.01200000e+03, ...,
         2.01200000e+03, 2.01200000e+03, 2.01200000e+03],
        [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        ...,
        [2.05000697e-01, 2.17916734e-01, 2.17916734e-01, ...,
         2.17916734e-01, 2.19241456e-01, 2.18910276e-01],
        [8.24267782e-01, 7.85216179e-01, 8.53556485e-01, ...,
         8.84239888e-01, 8.67503487e-01, 8.82845188e-01],
        [6.43067847e-01, 4.86725664e-01, 6.87315634e-01, ...,
         6.87315634e-01, 6.72566372e-01, 7.22713864e-01]],

       [[2.01200000e+03, 2.01200000e+03, 2.01200000e+03, ...,
         2.01200000e+03, 2.01200000e+03, 2.01200000e+03],
        [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
         1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
        [1.00000000e+00, 

In [488]:
result.shape

(8784, 14, 77)

In [134]:
# normalized_data = np.zeros_like(result)
# for feature_index in range(result.shape[1]):  # 遍历每个特征
#     feature_values = result[:, feature_index, :]  # 当前特征的所有数据点
#     non_zero_mask = feature_values != 0  # 非零值掩码
#     non_zero_values = feature_values[non_zero_mask]  # 提取非零值

#     # 计算非零值的均值和标准差
#     mean = np.mean(non_zero_values)
#     std = np.std(non_zero_values)

#     # 只在非零值上应用 Z-score 标准化
#     if std > 0:
#         normalized_values = (non_zero_values - mean) / std
#         feature_values[non_zero_mask] = normalized_values  # 将计算结果赋值回相应位置
#     else:
#         # 如果所有非零值都相等（std=0），可以选择保留原值或其他处理
#         feature_values[non_zero_mask] = non_zero_values  # 这里选择保留原值

#     normalized_data[:, feature_index, :] = feature_values  # 更新标准化数据
# print("Normalized data shape:", normalized_data.shape)

Normalized data shape: (8784, 14, 93)


In [489]:
# train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70]
train_list=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
test_list=[52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]
# test_list=[71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98]

In [490]:
train_fusion_data=result[:,:,train_list]
test_fusion_data=result[:,:,test_list]

In [491]:
train_fusion_data = np.transpose(train_fusion_data, (0, 2, 1))
test_fusion_data = np.transpose(test_fusion_data, (0, 2, 1))
# train_fusion_data = np.reshape(train_fusion_data, (8784, 63,18))
# test_fusion_data = np.reshape(test_fusion_data, (8784, 40,18))

In [492]:
train_fusion_data.shape

(8784, 52, 14)

In [493]:
test_fusion_data.shape

(8784, 25, 14)

In [56]:
# train_data = np.reshape(train_data, (8784, 41,18))
# test_data = np.reshape(test_data, (8784, 62,18))

## Timestap

In [494]:
timestap_list=[0,1,2,3]

In [495]:
timestap=X_raw[:,timestap_list,:]

In [496]:
timestap=timestap.astype(np.int32)
timestap=timestap.astype(np.str_)

In [497]:
for i in range(len(timestap)):
    for m in range(1,4):
        timestap[i,m,1]=timestap[i,m,1].zfill(2)

In [498]:
timestap=timestap[:,:,1]
timestap_result=[]
for i in range(len(timestap)):
    x=timestap[i,0]+timestap[i,1]+timestap[i,2]+'-'+timestap[i,3]
    timestap_result.append(x)

In [499]:
timestap_result=np.array(timestap_result)

In [500]:
timestap_result.shape

(8784,)

### invid_mask掩码

In [501]:
def generate1_random_array(length):
    array = []
    for _ in range(length):
        array.append(False)
    return np.array(array)
 
# 生成长度为10的随机数组
train_invid_mask=[]
test_invid_mask=[]
for i in range(len(timestap)):
    random_array = generate1_random_array(train_data.shape[1])
    random_array1 = generate1_random_array(test_data.shape[1])
    train_invid_mask.append(random_array)
    test_invid_mask.append(random_array1)
train_invid_mask=np.array(train_invid_mask)
test_invid_mask=np.array(test_invid_mask)

In [502]:
train_invid_mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

### test_mask掩码

In [503]:
import random
 
def generate_random_array(length):
    array = []
    for _ in range(length):
            array.append(random.choice([False,True]))
    return np.array(array)

# 生成长度为10的随机数组
test_mask=[]
random_array=generate_random_array(test_data.shape[1])
for i in range(len(timestap)):
    test_mask.append(random_array)
test_mask=np.array(test_mask)

### 地理位置

#### 创建r_pos_mat

In [504]:
stations = pd.read_csv("D:\Spatial_interpolation\SSIN\data\Station_info.csv")

In [505]:
stations.drop([  4,  15,  28,  34,  45,  47,  55,  60,  82,  83,  84,  85,  86,
        87,  88,  89,  90,  91,  92,  93,  94,  96,  98,  99, 100, 102],inplace=True)

In [506]:
df_train=stations.iloc[0:52]
df_test=stations.iloc[52:77]

In [507]:
def calc_dist_angle_mat(df, out_path):
    lons, lats = df["lon"].values, df["lat"].values
    dist_angle_mat = np.zeros((len(lons), len(lons), 2))

    for i in range(len(lons)):
        for j in range(len(lons)):
            dist = Geodesic.WGS84.Inverse(lats[i], lons[i], lats[j], lons[j])
            dist_angle_mat[i, j, 0] = dist["s12"] / 1000.0  # distance, km
            dist_angle_mat[i, j, 1] = dist["azi1"]  # azimuth at the first point in degrees

    print(dist_angle_mat.shape)
    # print(dist_angle_mat)
    np.save(out_path, dist_angle_mat)

In [508]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    # out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    out_dir = f"{base_dir}"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_train.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_train, out_path)

(52, 52, 2)


In [509]:
if __name__ == "__main__":
    base_dir = "../data"

    # HK dataset
    # info_path = f"{base_dir}/HK_123_data/hko_stations_info.csv"
    out_dir = f"{base_dir}"

    # BW dataset
    # info_path = f"{base_dir}/BW_132_data/BW_stations_info.csv"
    # out_dir = f"{base_dir}/BW_132_data"

    # os.makedirs(out_dir, exist_ok=True)

    out_name = "dist_angle_mat_test.npy"
    out_path = f"{out_dir}/{out_name}"
    calc_dist_angle_mat(df_test, out_path)

(25, 25, 2)


In [510]:
def generate_stat_from_train_data(train_info_df, r_pos_mat):
    # info_df = pd.read_csv(info_path)
    # ori_r_pos_mat = np.load(relative_pos_mat_path)

    # is_test = info_df["is_test"].values
    # train_mask = np.where(is_test == 0, True, False)
    # train_info_df = info_df.loc[train_mask, :]

    lat_mean, lat_std, lat_max, lat_min = train_info_df["lat"].mean(), train_info_df["lat"].std(ddof=0), \
                                          train_info_df["lat"].max(), train_info_df["lat"].min()
    lon_mean, lon_std, lon_max, lon_min = train_info_df["lon"].mean(), train_info_df["lon"].std(ddof=0), \
                                          train_info_df["lon"].max(), train_info_df["lon"].min()

    # indexes = np.where(train_mask)[0]
    # idx_i, idx_j = np.ix_(indexes, indexes)
    # r_pos_mat = ori_r_pos_mat[idx_i, idx_j, :]

    r_dist_mat = r_pos_mat[:, :, 0]
    r_angle_mat = r_pos_mat[:, :, 1]

    r_dist_mean, r_dist_std, r_dist_max, r_dist_min = np.mean(r_dist_mat), np.std(r_dist_mat), \
                                                      np.max(r_dist_mat), np.min(r_dist_mat),
    r_angle_mean, r_angle_std, r_angle_max, r_angle_min = np.mean(r_angle_mat), np.std(r_angle_mat), \
                                                          np.max(r_angle_mat), np.min(r_angle_mat)

    stat_dict = {}
    stat_dict["lat_mean"], stat_dict["lat_std"], stat_dict["lat_max"], stat_dict["lat_min"] = \
        lat_mean, lat_std, lat_max, lat_min
    stat_dict["lon_mean"], stat_dict["lon_std"], stat_dict["lon_max"], stat_dict["lon_min"] = \
        lon_mean, lon_std, lon_max, lon_min

    stat_dict["r_dist_mean"], stat_dict["r_dist_std"], stat_dict["r_dist_max"], stat_dict["r_dist_min"] = \
        r_dist_mean, r_dist_std, r_dist_max, r_dist_min
    stat_dict["r_angle_mean"], stat_dict["r_angle_std"], stat_dict["r_angle_max"], stat_dict["r_angle_min"] = \
        r_angle_mean, r_angle_std, r_angle_max, r_angle_min

    print("Calculates the statistics of training data. Done!")

    # with open("./data/hk_data_stats.pkl".format(out_name), "wb") as fp:
    #     pickle.dump(stat_dict, fp)

    return stat_dict

In [511]:
train_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_train.npy")
test_df=np.load("D:\Spatial_interpolation\data\dist_angle_mat_test.npy")

In [512]:
stat_dict=generate_stat_from_train_data(df_train, train_df)

Calculates the statistics of training data. Done!


In [513]:

train_df[:, :, 0] = (train_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
train_df[:, :, 1] = (train_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

In [514]:
test_df[:, :, 0] = (test_df[:, :, 0] - stat_dict["r_dist_mean"]) / stat_dict["r_dist_std"]
test_df[:, :, 1] = (test_df[:, :, 1] - stat_dict["r_angle_mean"]) / stat_dict["r_angle_std"]

### 创建edge_index

In [515]:
def haversine(lat1, lon1, lat2, lon2):
    # 将经纬度从度转换为弧度
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine 公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # 地球半径的平均半径，单位公里
    return km

In [516]:
def compute_distance_matrix(latitudes, longitudes):
    num_sites = len(latitudes)
    distance_matrix = np.zeros((num_sites, num_sites))
    
    for i in range(num_sites):
        for j in range(i + 1, num_sites):
            dist = haversine(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # 因为距离是对称的

    return distance_matrix

In [517]:
def create_edge_index_from_nearest_neighbors(distance_matrix, num_neighbors=3):
    num_sites = distance_matrix.shape[0]
    edge_index = []

    for i in range(num_sites):
        # 获取与站点 i 距离最近的 num_neighbors 个站点的索引
        neighbors_indices = np.argsort(distance_matrix[i])[:num_neighbors + 1]  # +1 因为最近的包括自己
        for j in neighbors_indices:
            if i != j:  # 避免自连接
                edge_index.append([i, j])

    return torch.tensor(edge_index).t().contiguous()

In [518]:
longitudes_train,latitudes_train=df_train["lon"].values, df_train["lat"].values
# 计算距离矩阵
distance_matrix_train = compute_distance_matrix(longitudes_train, latitudes_train)

# 生成边索引
edge_index_train = create_edge_index_from_nearest_neighbors(distance_matrix_train, num_neighbors=3)





In [519]:
longitudes_test,latitudes_test=df_test["lon"].values, df_test["lat"].values
# 计算距离矩阵
distance_matrix_test = compute_distance_matrix(longitudes_test, latitudes_test)

# 生成边索引
edge_index_test = create_edge_index_from_nearest_neighbors(distance_matrix_test, num_neighbors=3)

### 创建最后的结果

In [520]:
test={"test_data":test_data,"fusion_data":test_fusion_data,"invalid_masks":test_invid_mask,"test_masks":test_mask,"r_pos_mat":test_df,"timestamps":timestap_result,"edg_index":edge_index_test}

In [521]:
train={"train_data":train_data,"fusion_data":train_fusion_data,"invalid_masks":train_invid_mask,"r_pos_mat":train_df,"timestamps":timestap_result,"edg_index":edge_index_train}

In [522]:
print(list(test.keys()))

['test_data', 'fusion_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps', 'edg_index']


In [523]:
print(list(train.keys()))

['train_data', 'fusion_data', 'invalid_masks', 'r_pos_mat', 'timestamps', 'edg_index']


In [525]:
with open('train.pkl', 'wb') as f:
    pickle.dump(train, f)


with open('test.pkl', 'wb') as f:
    pickle.dump(test, f)

### test

In [34]:
# import pandas as pd

# data = pd.read_pickle("2012-2014_data_train.pkl")


In [6]:
# data_bw_train = pd.read_pickle("2012-2014_data_bw_train.pkl")
# data_bw_test = pd.read_pickle("2012-2014_data_bw_test.pkl")

In [17]:
# data.keys()

dict_keys(['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps'])

In [18]:
# data_bw_train.keys()

dict_keys(['test_data', 'invalid_masks', 'test_masks', 'r_pos_mat', 'timestamps'])

In [26]:
# data_bw_train["r_pos_mat"].shape

(132, 132, 2)

In [36]:
# data["invalid_masks"].shape

(8784, 41)

In [526]:
import pickle
import numpy as np
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)

train=data["train_data"]
train[0]
train=data["fusion_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape
train_data=train_data_expanded[:,5:,:,:]
train_data.shape

(8784, 9, 52, 1)

In [92]:
with open('2012-2014_data.pkl', 'rb') as file:
    data = pickle.load(file)

data.keys()

dict_keys(['train_data', 'invalid_masks', 'timestamps', 'r_pos_mat'])

In [93]:
temp=data["train_data"]
temp.shape

(3640, 106, 1)

In [96]:
train.shape

(8784, 63, 1)

In [97]:
train[0]

array([[  0.      ],
       [221.      ],
       [185.      ],
       [267.      ],
       [119.      ],
       [303.      ],
       [313.      ],
       [ 73.      ],
       [ 81.      ],
       [119.      ],
       [ 91.      ],
       [102.      ],
       [196.      ],
       [240.      ],
       [133.      ],
       [ 72.      ],
       [138.      ],
       [135.      ],
       [192.      ],
       [155.      ],
       [206.      ],
       [177.      ],
       [ 87.      ],
       [110.      ],
       [ 66.      ],
       [ 87.      ],
       [ 80.      ],
       [ 71.      ],
       [ 69.      ],
       [296.      ],
       [344.      ],
       [301.      ],
       [291.      ],
       [154.      ],
       [  0.      ],
       [312.      ],
       [210.      ],
       [280.      ],
       [  0.      ],
       [310.      ],
       [350.      ],
       [340.      ],
       [360.      ],
       [340.      ],
       [205.53244 ],
       [140.      ],
       [282.      ],
       [100. 

In [94]:
temp[0]

array([[0.4],
       [2.9],
       [0.2],
       [0. ],
       [0. ],
       [1.4],
       [1.3],
       [1. ],
       [0. ],
       [0.7],
       [0.9],
       [3.4],
       [0. ],
       [0.4],
       [0.8],
       [5.6],
       [0. ],
       [5. ],
       [0.1],
       [0.2],
       [0.6],
       [0.6],
       [0.2],
       [0.1],
       [3.3],
       [0. ],
       [0.1],
       [0.1],
       [0. ],
       [1.6],
       [0. ],
       [0. ],
       [0.3],
       [0.1],
       [1. ],
       [0.2],
       [0.2],
       [1. ],
       [1.9],
       [0.8],
       [2. ],
       [0. ],
       [0. ],
       [0.1],
       [0. ],
       [0.4],
       [0.5],
       [2.1],
       [0.9],
       [0. ],
       [0. ],
       [0.5],
       [1.7],
       [0.2],
       [0. ],
       [0. ],
       [0.1],
       [0.3],
       [0.2],
       [2.6],
       [0. ],
       [0. ],
       [0.4],
       [1.1],
       [0.1],
       [3.7],
       [1. ],
       [0. ],
       [0. ],
       [0. ],
       [0. ],
      

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class ModifiedAFFWithSoftmax(nn.Module):
    def __init__(self, num_timestamps=8784, num_features=9, num_sites=103, r=4):
        super(ModifiedAFFWithSoftmax, self).__init__()
        self.num_timestamps = num_timestamps
        self.num_features = num_features
        self.num_sites = num_sites
        inter_channels = int(num_features // r * num_features)
        
        # 图卷积网络

        self.GCN1=GCNConv(num_timestamps * num_features, inter_channels),
        self.batch=nn.BatchNorm2d(inter_channels),
        self.relu1=nn.ReLU(inplace=True),
        self.GCN2=GCNConv(inter_channels, num_timestamps * num_features),
        self.relu2=nn.BatchNorm2d(num_timestamps * num_features),

        
        # 传统卷积部分调整为一维操作，以处理1D特征数据
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Conv1d(num_features, inter_channels, kernel_size=1,stride=1, padding=0),
            nn.BatchNorm1d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv1d(inter_channels, num_features, kernel_size=1,stride=1, padding=0),
            nn.BatchNorm1d(num_features),
        )

        self.mlp=nn.Linear(2,1)


    def forward(self, x, edge_index):
        # 调整x的形状以适应图卷积网络的输入
        x=x.squeeze(-1).permute(2, 1, 0)
        x_gcn = x.reshape(self.num_sites, -1)  # (103, 8784*9)

        # GCN 处理
        x_gcn=self.GCN1(x_gcn, edge_index)
        x_gcn=self.batch(x_gcn)
        x_gcn=self.relu1(x_gcn)
        x_gcn=self.GCN2(x_gcn, edge_index)
        xl=self.relu2(x_gcn)
        
        # 调整为原始时间戳维度
        x_gcn = x_gcn.view(self.num_sites,  self.num_features,self.num_timestamps).permute(2, 1, 0).unsqueeze(-1)  # (8784, 9, 103, 1)

        # Global attention 处理
        xg = self.global_att(x)
        repeat_num=xl.size(2)
        xg=xg.repeat(1, 1, repeat_num, 1)
        xlg = torch.cat((xl, xg), dim=3)
        batch_size, num_channels, height, width = xlg.size()
        xlg = xlg.view(-1, width)
        xlg = F.relu(self.mlp(xlg))
        xlg=xlg.view(batch_size,num_channels,height,-1)
        # x_global = x.squeeze(-1).permute(0, 2, 1)  # 形状变为 (8784, 103, 9)
        # x_global = self.global_att(x_global)  # 形状: (8784, 9, 103, 1)
        # x_global = x_global.permute(0, 2, 1).unsqueeze(-1)

        # xl = self.local_att(x)
        # xg = self.global_att(x)
        # repeat_num=xl.size(2)
        # xg=xg.repeat(1, 1, repeat_num, 1)
        # xlg = torch.cat((xl, xg), dim=3)
        # batch_size, num_channels, height, width = xlg.size()
        # xlg = xlg.view(-1, width)
        # xlg = F.relu(self.mlp(xlg))
        # xlg=xlg.view(batch_size,num_channels,height,-1)

        # Combine and process weights
        # x_combined = x_gcn + x_global
        weights = xlg.view(xlg.size(0), self.num_features, -1)
        weights = F.softmax(weights, dim=1)
        weights = weights.view_as(xlg)
        
        separated_weights = torch.chunk(weights, self.num_features, dim=1)
        weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
        output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

        return output.squeeze(1)


### ceshi 

In [558]:
import pickle
import numpy as np
with open('2012-2014_data_train.pkl', 'rb') as file:
    data = pickle.load(file)

# train=data["fusion_data"]
# train[0]
# train=data["fusion_data"]
# train_data_expanded = np.expand_dims(train, axis=-1)
# train_data_expanded.shape
# train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
# train_data_expanded.shape
# train_data=train_data_expanded[:,5:,:,:]
# edg_index_in=data["edg_index"]


In [562]:
data["train_data"].shape

(3640, 106, 1)

In [569]:
data["train_data"].shape

(3640, 106, 1)

In [580]:
import pickle
import numpy as np
with open('2012-2014_data_test.pkl', 'rb') as file:
    test = pickle.load(file)

In [584]:
test["test_masks"][4]

array([ True, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True,  True, False,
        True, False, False, False, False, False, False, False, False,
        True,  True, False,  True, False, False, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False,

## ceshi

In [544]:
train_data.shape

(8784, 9, 52, 1)

In [545]:
edg_index_in.shape

torch.Size([2, 156])

In [2]:
import pickle
import numpy as np
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)

train=data["fusion_data"]
train[0]
train=data["fusion_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape
train_data=train_data_expanded[:,5:,:,:]
edg_index_in=data["edg_index"]

### 测试用例

In [5]:
import pickle
import numpy as np
with open('train.pkl', 'rb') as file:
    data = pickle.load(file)

train=data["fusion_data"]
train[0]
train=data["fusion_data"]
train_data_expanded = np.expand_dims(train, axis=-1)
train_data_expanded.shape
train_data_expanded = np.transpose(train_data_expanded, (0, 2, 1,3))
train_data_expanded.shape
train_data=train_data_expanded[:,5:,:,:]
edg_index_in=data["edg_index"]
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class ModifiedAFFWithSoftmax(nn.Module):
    def __init__(self, num_timestamps=8784, num_features=9, r=4):
        super(ModifiedAFFWithSoftmax, self).__init__()
        self.num_timestamps = num_timestamps
        self.num_features = num_features
        # self.num_sites = num_sites
        inter_channels = int(num_features // r * num_timestamps)
        
        # 图卷积网络

        self.GCN1=GCNConv(num_timestamps * num_features, inter_channels)
        self.batch1=nn.BatchNorm1d(inter_channels)
        self.relu1=nn.ReLU(inplace=True)
        self.GCN2=GCNConv(inter_channels, num_timestamps * num_features)
        self.batch2=nn.BatchNorm1d(num_timestamps * num_features)
        # self.relu2=nn.ReLU(inplace=True)
        
        # 传统卷积部分调整为一维操作，以处理1D特征数据
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(num_features, inter_channels, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(inter_channels, num_features, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(num_features),   
        )

        self.mlp=nn.Linear(2,1)


    def forward(self, x, edge_index):
        # 调整x的形状以适应图卷积网络的输入
        num_sites=x.size(2)
        x_gcn=x
        x_gcn=x_gcn.squeeze(-1).permute(2, 1, 0)
        x_gcn = x_gcn.reshape(num_sites, -1)  # (103, 8784*9)


        # GCN 处理
        x_gcn=self.GCN1(x_gcn, edge_index)
        x_gcn=self.batch1(x_gcn)
        x_gcn=self.relu1(x_gcn)
        x_gcn=self.GCN2(x_gcn, edge_index)
        xl=self.batch2(x_gcn)
        # xl=self.relu2(x_gcn)
        
        # 调整为原始时间戳维度
        xl = xl.view(num_sites,  self.num_features,self.num_timestamps).permute(2, 1, 0).unsqueeze(-1)  # (8784, 9, 103, 1)

        # Global attention 处理
        xg = self.global_att(x)
        repeat_num=xl.size(2)
        xg=xg.repeat(1, 1, repeat_num, 1)
        xlg = torch.cat((xl, xg), dim=3)
        batch_size, num_channels, height, width = xlg.size()
        xlg = xlg.view(-1, width)
        xlg = F.relu(self.mlp(xlg))
        xlg=xlg.view(batch_size,num_channels,height,-1)

        weights = xlg.view(xlg.size(0), self.num_features, -1)
        weights = F.softmax(weights, dim=1)
        weights = weights.view_as(xlg)
        
        separated_weights = torch.chunk(weights, self.num_features, dim=1)
        weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
        output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

        return output.squeeze(1)
# class ModifiedAFFWithSoftmax(nn.Module):
#     def __init__(self, channels=9, r=4, num_features=9):
#         super(ModifiedAFFWithSoftmax, self).__init__()
#         self.num_features = num_features
#         inter_channels = int(channels // r )
        
        
#         self.local_att = nn.Sequential(
#             nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(inter_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(channels),
#         )
        
#         self.global_att = nn.Sequential(
#             nn.AdaptiveAvgPool2d(1),
#             nn.Conv2d(channels, inter_channels, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(inter_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(inter_channels, channels, kernel_size=1, stride=1, padding=0),
#             nn.BatchNorm2d(channels),   
#         )

#         self.mlp=nn.Linear(2,1)
#     def forward(self, x):
#         xl = self.local_att(x)
#         xg = self.global_att(x)
#         repeat_num=xl.size(2)
#         xg=xg.repeat(1, 1, repeat_num, 1)
#         xlg = torch.cat((xl, xg), dim=3)
#         batch_size, num_channels, height, width = xlg.size()
#         xlg = xlg.view(-1, width)
#         xlg = F.relu(self.mlp(xlg))
#         xlg=xlg.view(batch_size,num_channels,height,-1)
#         # xlg = xl + xg
#         # xlg = xlg - xl
        
#         weights = xlg.view(xlg.size(0), self.num_features, -1)
#         weights = F.softmax(weights, dim=1)
#         weights = weights.view_as(xlg)
        
#         separated_weights = torch.chunk(weights, self.num_features, dim=1)
#         weighted_features = [x[:, i:i+1, :, :] * separated_weights[i] for i in range(self.num_features)]
        
#         output = torch.sum(torch.stack(weighted_features, dim=0), dim=0)

#         return output.squeeze(1)
# 初始化模型
model = ModifiedAFFWithSoftmax()

# 生成模拟输入数据
train_tensor = torch.from_numpy(train_data)  # Batch size 8784, 18 features, 41 sites, width 1
train_tensor = train_tensor.float()

# 前向传播
output_tensor = model(train_tensor,edg_index_in)

# 打印输出形状
print("Output shape:", output_tensor.shape)

Output shape: torch.Size([8784, 71, 1])


In [536]:
train_tensor.size()

torch.Size([8784, 9, 52, 1])

In [272]:
output_numpy=output_tensor.detach().cpu().numpy()

In [275]:
output_numpy[0]

array([[ 32.127617],
       [107.00171 ],
       [136.66765 ],
       [ 93.62737 ],
       [126.2027  ],
       [ 86.40228 ],
       [ 76.56961 ],
       [142.7038  ],
       [188.48268 ],
       [146.61589 ],
       [144.86035 ],
       [147.1432  ],
       [135.56206 ],
       [ 95.803604],
       [125.354324],
       [152.61635 ],
       [136.80568 ],
       [112.02063 ],
       [100.627556],
       [125.41099 ],
       [100.76597 ],
       [ 86.37686 ],
       [ 70.158936],
       [140.76495 ],
       [156.76047 ],
       [158.44688 ],
       [185.46153 ],
       [146.83614 ],
       [148.74702 ],
       [ 64.645706],
       [ 46.472363],
       [ 88.016426],
       [ 77.6937  ],
       [123.21856 ],
       [ 56.151684],
       [112.73174 ],
       [110.40742 ],
       [ 93.14596 ],
       [102.831436],
       [ 59.37419 ],
       [ 54.77721 ],
       [ 49.511074],
       [ 44.447273],
       [ 53.370037],
       [115.65097 ],
       [117.68736 ],
       [ 97.92604 ],
       [109.5

In [277]:
train=data["train_data"]

In [279]:
train[0]

array([[  0.      ],
       [221.      ],
       [185.      ],
       [267.      ],
       [119.      ],
       [303.      ],
       [313.      ],
       [ 73.      ],
       [ 81.      ],
       [119.      ],
       [ 91.      ],
       [102.      ],
       [196.      ],
       [240.      ],
       [133.      ],
       [ 72.      ],
       [138.      ],
       [135.      ],
       [192.      ],
       [155.      ],
       [206.      ],
       [177.      ],
       [ 87.      ],
       [110.      ],
       [ 66.      ],
       [ 87.      ],
       [ 80.      ],
       [ 71.      ],
       [ 69.      ],
       [296.      ],
       [344.      ],
       [301.      ],
       [291.      ],
       [154.      ],
       [  0.      ],
       [312.      ],
       [210.      ],
       [280.      ],
       [  0.      ],
       [310.      ],
       [350.      ],
       [340.      ],
       [360.      ],
       [340.      ],
       [205.53244 ],
       [140.      ],
       [282.      ],
       [100. 

In [99]:
train=data["fusion_data"]
train.shape

(8784, 63, 18)

In [40]:
def load_train_data(path):
        # load data
        with open(path, "rb") as fp:
            data_dict = pickle.load(fp)

        all_seq_data = data_dict["train_data"][:, :, 0:1]
        invalid_masks_data = data_dict["invalid_masks"]
        r_pos_mat_data = data_dict["r_pos_mat"]

        if "adj_attn_mask" in data_dict.keys():
            adj_attn_mask = data_dict["adj_attn_mask"]
        else:
            adj_attn_mask = None

        return all_seq_data, invalid_masks_data, r_pos_mat_data, adj_attn_mask

In [41]:
all_seq_data, invalid_masks_data, r_pos_mat_data, adj_attn_mask = load_train_data('train.pkl')
d_feat, d_pos = all_seq_data[0].shape[-1], r_pos_mat_data[0].shape[-1]

In [42]:
all_seq_data.shape

(8784, 63, 1)

In [44]:
invalid_masks_data.shape

(8784, 63)

In [45]:
r_pos_mat_data.shape

(63, 63, 2)

### creat data

In [ ]:
import torch
import random
import numpy as np
from tqdm import tqdm
def create_train_data(epoch, all_seq_data, invalid_masks, batch_size, masked_lm_prob, times=1, adj_attn_mask=None):
    """times: create how many times of the whole data"""

    data_num = len(all_seq_data)
    seq_len = all_seq_data.shape[1]
    all_indexes = list(range(data_num))

    max_pred_per_seq = int(seq_len * masked_lm_prob)

    batch_num = data_num // batch_size + 1

    with tqdm(total=batch_num*times, desc=f'Epoch {epoch}:') as pbar:
        for t in range(times):
            random.shuffle(all_indexes)  # for each masked round, shuffle the whole data

            for i in range(batch_num):
                end_idx = min(data_num, (i + 1) * batch_size)
                batch_indexes = all_indexes[i * batch_size: end_idx]

                masked_seq_list, masked_idx_list = [], []
                masked_labels_list, masked_label_weights_list, attn_mask_list = [], [], []
                for idx in batch_indexes:  # todo: modify batch, concatenate
                    seq_data = all_seq_data[idx]

                    if invalid_masks is not None:
                        invalid_mask = invalid_masks[idx]
                    else:
                        invalid_mask = None

                    masked_seq, masked_indexes, masked_labels, masked_label_weights, attn_mask = \
                        create_random_masked_data(seq_data, seq_len, invalid_mask, max_pred_per_seq, masked_lm_prob)

                    if adj_attn_mask is not None:
                        attn_mask = np.logical_and(attn_mask, adj_attn_mask).astype(int)

                    if masked_seq is not None:
                        masked_seq_list.append(masked_seq)
                        masked_idx_list.append(masked_indexes)
                        masked_labels_list.append(masked_labels)
                        masked_label_weights_list.append(masked_label_weights)
                        attn_mask_list.append(attn_mask)

                masked_seq_arr = np.array(masked_seq_list)
                masked_idx_arr = np.array(masked_idx_list)
                masked_labels_arr = np.array(masked_labels_list)
                masked_label_weights_arr = np.array(masked_label_weights_list)
                attn_mask_arr = np.array(attn_mask_list)

                # When all masked_seq with zero std, will generate empty batch
                # if masked_seq_arr.size != 0:
                yield [masked_seq_arr, masked_idx_arr, masked_labels_arr, masked_label_weights_arr, attn_mask_arr]
                pbar.update(1)


def create_random_masked_data(seq_data, seq_len, invalid_mask, max_pred_per_seq, masked_lm_prob=0.15):
    masked_seq = seq_data.copy()
    full_indexes = set(range(seq_len))

    if invalid_mask is not None:
        invalid_indexes = set(np.where(invalid_mask)[0])
    else:
        invalid_indexes = set()

    valid_indexes = full_indexes - invalid_indexes
    if len(valid_indexes) < 5:
        return None, None, None, None, None

    # masked_lm_prob = random.uniform(0.1, 0.3)
    n_pred = min(max_pred_per_seq, max(1, int(round(len(valid_indexes) * masked_lm_prob))))
    masked_indexes = random.sample((list(valid_indexes)), n_pred)
    masked_labels = list(seq_data[masked_indexes, :1])

    unmasked_indexes = list(valid_indexes - set(masked_indexes))
    mean_value = np.mean(seq_data[unmasked_indexes, :1])
    std_value = np.std(seq_data[unmasked_indexes, :1])

    # if std_value == 0:  # Discard the seqs: after normalization, will generate nan or inf value
    #     return None, None, None, None, None
    # masked_seq = (masked_seq - mean_value) / std_value  # normalization
    # masked_seq[masked_indexes, :1] = 0  # set masked nodes as 0

    # fixme: use this all zero sequence
    if std_value == 0:  # Discard the seqs: after normalization, will generate nan or inf value
        masked_seq = masked_seq - mean_value  # normalization
    else:
        masked_seq = (masked_seq - mean_value) / std_value  # normalization
    masked_seq[masked_indexes, :1] = 0  # set masked nodes as 0; this 0 denotes the mean value

    masked_label_weights = [1.0] * len(masked_labels)
    if max_pred_per_seq > n_pred:
        n_pad = max_pred_per_seq - n_pred
        masked_labels.extend([[0]] * n_pad)
        masked_indexes.extend([0] * n_pad)
        masked_label_weights.extend([0] * n_pad)

    attn_mask = get_attn_mask(seq_len, unmasked_indexes)

    masked_indexes = np.array(masked_indexes).astype(int)
    masked_label_weights = np.array(masked_label_weights).astype(float)
    attn_mask = np.array(attn_mask).astype(int)

    # fixme: std = 0
    # masked_labels = (np.array(masked_labels).astype(float) - mean_value) / std_value  # standardize label
    if std_value == 0:  # Discard the seqs: after normalization, will generate nan or inf value
        masked_labels = np.array(masked_labels).astype(float) - mean_value  # standardize label
    else:
        masked_labels = (np.array(masked_labels).astype(float) - mean_value) / std_value  # standardize label

    return masked_seq, masked_indexes, masked_labels, masked_label_weights, attn_mask
def get_attn_mask(max_seq_len, unmasked_indexes):
    attn_vec = np.zeros(max_seq_len)

    # for each node, cut off the edge between it and invalid (masked or padded) nodes
    attn_vec[unmasked_indexes] = 1  # vector
    attn_mask = np.tile(attn_vec, (max_seq_len, 1))  # n*n matrix:

    # for each node (mainly for masked nodes), add itself edge for attention calculation
    eye_mask = np.eye(max_seq_len)

    attn_mask = np.logical_or(attn_mask, eye_mask).astype(int)

    return attn_mask

In [ ]:
def load_test_data_generator(self):
        # load data
        with open(self.args.test_data_path, "rb") as fp:
            data_dict = pickle.load(fp)

        all_seq_data = data_dict["test_data"][:, :, 0:1]
        r_pos_mat = data_dict["r_pos_mat"]
        invalid_masks = data_dict["invalid_masks"]
        test_masks = data_dict["test_masks"]
        all_timestamps = data_dict["timestamps"]

        if "adj_attn_mask" in data_dict.keys():
            adj_attn_mask = data_dict["adj_attn_mask"]
        else:
            adj_attn_mask = None

        test_data_generator = create_test_data(all_seq_data, invalid_masks, test_masks, all_timestamps, adj_attn_mask)

        return r_pos_mat, test_data_generator

### gcn fusion 模块

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class ModifiedAFFWithSoftmax(nn.Module):
    def __init__(self, num_timestamps=8784, num_features=9, num_sites=103, r=4):
        super(ModifiedAFFWithSoftmax, self).__init__()
        self.num_timestamps = num_timestamps
        self.num_features = num_features
        self.num_sites = num_sites
        inter_channels = int(num_features // r * num_sites)
        
        # 图卷积网络
        self.gcn1 = GCNConv(num_timestamps * num_features, inter_channels)
        self.gcn2 = GCNConv(inter_channels, num_timestamps)
        
        # 传统卷积部分调整为一维操作，以处理1D特征数据
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Conv1d(num_features, inter_channels, kernel_size=1),
            nn.BatchNorm1d(inter_channels),
            nn.ReLU(inplace=True),
            nn.Conv1d(inter_channels, num_features, kernel_size=1),
            nn.BatchNorm1d(num_features),
        )

    def forward(self, x, edge_index):
        # 调整x的形状以适应图卷积网络的输入
        x_gcn = x.permute(2, 0, 1).reshape(self.num_sites, -1)  # (103, 8784*9)

        # GCN 处理
        x_gcn = F.relu(self.gcn1(x_gcn, edge_index))
        x_gcn = self.gcn2(x_gcn, edge_index)  # 输出形状: (103, 8784)
        
        # 调整为原始时间戳维度
        x_gcn = x_gcn.view(self.num_sites, self.num_timestamps, self.num_features).permute(1, 2, 0).unsqueeze(-1)  # (8784, 9, 103, 1)

        # Global attention 处理
        x_global = x.squeeze(-1).permute(0, 2, 1)  # 形状变为 (8784, 103, 9)
        x_global = self.global_att(x_global)  # 形状: (8784, 9, 103, 1)
        x_global = x_global.permute(0, 2, 1).unsqueeze(-1)

        # Combine and process weights
        x_combined = x_gcn + x_global
        weights = F.softmax(x_combined.squeeze(-1), dim=2)  # Softmax over sites dimension
        output = weights * x  # Element-wise multiplication for weighting
        output = output.sum(dim=2, keepdim=True)  # Sum over features

        return output  # 形状为 (8784, 103, 1)
